In [3]:
# Use a pipeline as a high-level helper
from transformers import pipeline

messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe = pipeline("text-generation", model="meta-llama/Llama-3.2-1B-Instruct")
pipe(messages)

Device set to use mps:0


RuntimeError: MPS backend out of memory (MPS allocated: 6.76 GB, other allocations: 628.00 KB, max allowed: 6.77 GB). Tried to allocate 64.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

model_name = "meta-llama/Llama-3.2-1B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",  # Automatically maps layers to CPU or GPU
    offload_folder="./offload"  # Folder for offloaded layers
).to(device)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/jack.yangjy/.pyenv/versions/3.10.11/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/jack.yangjy/.pyenv/versions/3.10.11/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/jack.yangjy/projects/youtube_summarizer/venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/jack.yangjy/projects/youtube_summarizer/venv/lib/python3.10/site-

In [3]:
link = 'https://www.youtube.com/watch?v=139UPjoq7Kw&t=25s&ab_channel=JaneStreet'
from data.fetch_transcript import fetch_transcript

output = fetch_transcript(link)
transcript = output['transcript']

from data.preprocess import clean_transcript, chunk_transcript

cleaned = clean_transcript(transcript)
chunks = chunk_transcript(cleaned)

Language: English (United States)
Language: en-US
Generated automatically: False
Translation available: True
-----


In [ ]:
prompt = "What is the capital of France?"
inputs = tokenizer(prompt, return_tensors="pt")
inputs = inputs.to(device)

with torch.inference_mode():
    outputs = model.generate(**inputs, max_new_tokens=50, eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
/Users/jack.yangjy/projects/youtube_summarizer/venv/lib/python3.10/site-packages/transformers/pytorch_utils.py:333: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)


What is the capital of France? Paris is not the capital of France, although it is the most populous city. The capital of France is actually Paris. Paris is the capital of the Île-de-France region and is located in the north of France. The city of Paris is


In [11]:
import replicate

def llama3_8b(prompt):
    output = replicate.run(
      "meta/meta-llama-3-8b-instruct",
      input={"prompt": prompt}
    )
    return ''.join(output)

In [13]:
# prompt = "What is the capital of France?"
prompt = chunks[0]
llama3_8b(prompt)

ReplicateError: ReplicateError Details:
title: Unauthenticated
status: 401
detail: You did not pass an authentication token